In [78]:
##Assignment 3
from langchain.chat_models import ChatOpenAI
from langchain.prompts.chat import (
    ChatPromptTemplate,
    HumanMessagePromptTemplate,
)
from langchain_core.messages import (
    SystemMessage,
    HumanMessage,
    AIMessage,
)
from langchain.prompts import MessagesPlaceholder
from langchain.memory import ConversationBufferMemory
from langchain.schema.runnable import RunnablePassthrough


# 1. Create LLM
llm = ChatOpenAI(temperature=0.1)

# 2. Memory setup for tracking chat history
memory = ConversationBufferMemory(
    return_messages=True
)

In [79]:
#3. Few-shot examples as actual chat messages
few_shot_messages = [
    HumanMessage(content="Top Gun"),
    AIMessage(content="🛩️👨‍✈️🔥"),
    HumanMessage(content="The Godfather"),
    AIMessage(content="👨‍👨‍👦🔫🍝"),
    HumanMessage(content="Finding Nemo"),
    AIMessage(content="🐠🌊👨‍👦"),
]

In [80]:
# 4. Full prompt using ChatPromptTemplate
prompt = ChatPromptTemplate.from_messages([
    SystemMessage(content="You are a helpful assistant that responds with exactly 3 emojis for any movie title."),
    *few_shot_messages,
    MessagesPlaceholder(variable_name="history"),
    HumanMessagePromptTemplate.from_template("{question}")  # ← this is KEY
])

In [81]:
#5. Memory Loader function
def load_memory(_):
    return memory.load_memory_variables({}).get("history", [])

#6. LCEL chain
chain = RunnablePassthrough.assign(history=load_memory) | prompt | llm

#7. Chain invocation + saving result in memory
def invoke_chain(question):
    result = chain.invoke({"question": question})
    memory.save_context({"input": question}, {"output": result.content})
    print("🤖", result.content)

In [82]:
#Invoke Function
invoke_chain("Inception")
print(memory.load_memory_variables({}))



🤖 🧠🌀🕰️
{'history': [HumanMessage(content='Inception', additional_kwargs={}, response_metadata={}), AIMessage(content='🧠🌀🕰️', additional_kwargs={}, response_metadata={})]}


In [83]:
invoke_chain("Titanic")
invoke_chain("Interstellar")
print(memory.load_memory_variables({}))


🤖 🚢❄️💔
🤖 🚀🌌⏰
{'history': [HumanMessage(content='Inception', additional_kwargs={}, response_metadata={}), AIMessage(content='🧠🌀🕰️', additional_kwargs={}, response_metadata={}), HumanMessage(content='Titanic', additional_kwargs={}, response_metadata={}), AIMessage(content='🚢❄️💔', additional_kwargs={}, response_metadata={}), HumanMessage(content='Interstellar', additional_kwargs={}, response_metadata={}), AIMessage(content='🚀🌌⏰', additional_kwargs={}, response_metadata={})]}
